<a href="https://colab.research.google.com/github/GolfSuradej/Python_Blind_Estimation_of_Room-acoustic-parameters/blob/main/Estimating_ThTt_ExtendedRIR_RevSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import glob
import h5py
from scipy.io import loadmat
# --- Signal Processing Libary
import scipy.io as sio
from scipy import signal
from scipy.signal import butter, hilbert, filtfilt
from scipy.fftpack import fft
from scipy.io import wavfile
import scipy
import IPython.display as ipd

# --- Machine Learining
import keras
#from random import shuffle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dropout, Activation
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from keras.models import model_from_json
from keras.preprocessing import sequence
from keras.models import load_model
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import  LSTM, RepeatVector, TimeDistributed,GlobalAveragePooling1D
from keras.models import Sequential
from keras.layers import BatchNormalization

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'keras.layers.convolutional'

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pwd

'/content'

In [ ]:
cd gdrive/MyDrive/training/

/content/gdrive/MyDrive/training


In [ ]:


# Prevent exhausting all the gpu memories.

X = np.load('./dataset/TrainingExt_TAEs.npy')
Y = np.load('./dataset/TrainingExt_ThT0Tt.npy')

x_train, y_train = shuffle(X, Y)
# split to 7 bands
x_train = x_train.astype('float32')
x_train_1 = x_train[:, 0, :]
x_train_2 = x_train[:, 1, :]
x_train_3 = x_train[:, 2, :]
x_train_4 = x_train[:, 3, :]
x_train_5 = x_train[:, 4, :]
x_train_6 = x_train[:, 5, :]
x_train_7 = x_train[:, 6, :]

y_train = y_train.astype('float32')
y_train_1 = y_train[:, :, :]
y_train_2 = y_train_1
y_train_3 = y_train_1
y_train_4 = y_train_1
y_train_5 = y_train_1
y_train_6 = y_train_1
y_train_7 = y_train_1
# y_train_2 = y_train[:, 1, :]
# y_train_3 = y_train[:, 2, :]
# y_train_4 = y_train[:, 3, :]
# y_train_5 = y_train[:, 4, :]
# y_train_6 = y_train[:, 5, :]
# y_train_7 = y_train[:, 6, :]

# Resharpe
X_train_1 = np.reshape(x_train_1, [len(x_train), 400, 1])
Y_train_1 = np.reshape(y_train_1, [len(y_train), 3, 1])
X_train_2 = np.reshape(x_train_2, [len(x_train), 400, 1])
Y_train_2 = np.reshape(y_train_2, [len(y_train), 3, 1])
X_train_3 = np.reshape(x_train_3, [len(x_train), 400, 1])
Y_train_3 = np.reshape(y_train_3, [len(y_train), 3, 1])
X_train_4 = np.reshape(x_train_4, [len(x_train), 400, 1])
Y_train_4 = np.reshape(y_train_4, [len(y_train), 3, 1])
X_train_5 = np.reshape(x_train_5, [len(x_train), 400, 1])
Y_train_5 = np.reshape(y_train_5, [len(y_train), 3, 1])
X_train_6 = np.reshape(x_train_6, [len(x_train), 400, 1])
Y_train_6 = np.reshape(y_train_6, [len(y_train), 3, 1])
X_train_7 = np.reshape(x_train_7, [len(x_train), 400, 1])
Y_train_7 = np.reshape(y_train_7, [len(y_train), 3, 1])

DNN_1 = Sequential()
DNN_1.add(Conv1D(filters=64, kernel_size=20, strides=1, name='Input_CNN1D', input_shape=(400, 1)))
DNN_1.add(Activation('relu'))
DNN_1.add(Conv1D(32, 5, strides=1, activation='relu'))
DNN_1.add(MaxPooling1D(2))
DNN_1.add(Conv1D(16, 5, activation='relu'))
DNN_1.add(MaxPooling1D(2))
DNN_1.add(Conv1D(8, 5, activation='relu'))
DNN_1.add(MaxPooling1D(2))
DNN_1.add(Dropout(0.2))
DNN_1.add(Flatten())
DNN_1.add(Dense(3, activation='relu'))  # Th and Tt parameters of the extended RIR model
print(DNN_1.summary())

keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.01,
    epsilon=1e-07,
    centered=False,
    name="RMSprop"
)
opt = keras.optimizers.RMSprop()

DNN_1.compile(opt, loss='mse', metrics=['mean_squared_error'])

epochs = 200
batch_size = 128
seed = 7

DNN_2 = DNN_1
DNN_3 = DNN_1
DNN_4 = DNN_1
DNN_5 = DNN_1
DNN_6 = DNN_1
DNN_7 = DNN_1

history = DNN_1.fit(X_train_1, Y_train_1, validation_split=0.3, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

history2 = DNN_2.fit(X_train_2, Y_train_2,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
history3 = DNN_3.fit(X_train_3, Y_train_3,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
history4 = DNN_4.fit(X_train_4, Y_train_4,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
history5 = DNN_5.fit(X_train_5, Y_train_5,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
history6 = DNN_6.fit(X_train_6, Y_train_6,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
history7 = DNN_7.fit(X_train_7, Y_train_7,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

DNN_1.save("MTFbasedCNN_ext_1.h5")
DNN_2.save("MTFbasedCNN_ext_2.h5")
DNN_3.save("MTFbasedCNN_ext_3.h5")
DNN_4.save("MTFbasedCNN_ext_4.h5")
DNN_5.save("MTFbasedCNN_ext_5.h5")
DNN_6.save("MTFbasedCNN_ext_6.h5")
DNN_7.save("MTFbasedCNN_ext_7.h5")
print("The fucking model has been save!!")


FileNotFoundError: [Errno 2] No such file or directory: './dataset/TrainingExt_TAEs.npy'

In [ ]:
cd ExtendedSP/

/content/gdrive/MyDrive/training/ExtendedSP


In [ ]:
ls

Training_TAEsExt.npy  Training_ThTts.npy  ver1_Tt_only/


In [ ]:
#load dataset
X = np.load('Training_TAEsExt.npy')
Y = np.load('Training_ThTts.npy')
print(X.shape)
print(Y.shape)

(51584, 7, 200)
(51584, 7, 2)


In [ ]:
Y[0]

array([[0.005   , 0.277125],
       [0.005   , 0.249375],
       [0.005   , 0.2565  ],
       [0.005   , 0.259125],
       [0.005   , 0.246375],
       [0.005   , 0.250125],
       [0.005   , 0.25575 ]])

In [ ]:
#Shuffle the trainingSet
x_train , y_train = shuffle(X , Y)

In [ ]:
# split to 7 bands
x_train = x_train.astype('float32')
x_train_1 = x_train[:,0,:]
x_train_2 = x_train[:,1,:]
x_train_3 = x_train[:,2,:]
x_train_4 = x_train[:,3,:]
x_train_5 = x_train[:,4,:]
x_train_6 = x_train[:,5,:]
x_train_7 = x_train[:,6,:]

y_train = y_train.astype('float32')
y_train_1 = y_train[:,0,:]
y_train_2 = y_train[:,1,:]
y_train_3 = y_train[:,2,:]
y_train_4 = y_train[:,3,:]
y_train_5 = y_train[:,4,:]
y_train_6 = y_train[:,5,:]
y_train_7 = y_train[:,6,:]

# Resharpe
X_train_1 = np.reshape(x_train_1,[len(x_train),200,1])
Y_train_1 = np.reshape(y_train_1,[len(y_train),2,1])
X_train_2 = np.reshape(x_train_2,[len(x_train),200,1])
Y_train_2 = np.reshape(y_train_2,[len(y_train),2,1])
X_train_3 = np.reshape(x_train_3,[len(x_train),200,1])
Y_train_3 = np.reshape(y_train_3,[len(y_train),2,1])
X_train_4 = np.reshape(x_train_4,[len(x_train),200,1])
Y_train_4 = np.reshape(y_train_4,[len(y_train),2,1])
X_train_5 = np.reshape(x_train_5,[len(x_train),200,1])
Y_train_5 = np.reshape(y_train_5,[len(y_train),2,1])
X_train_6 = np.reshape(x_train_6,[len(x_train),200,1])
Y_train_6 = np.reshape(y_train_6,[len(y_train),2,1])
X_train_7 = np.reshape(x_train_7,[len(x_train),200,1])
Y_train_7 = np.reshape(y_train_7,[len(y_train),2,1])

In [ ]:
DNN_1 = Sequential()
DNN_1.add(Conv1D(filters = 64, kernel_size = 20, strides = 1, name='Input_CNN1D', input_shape=(200,1)))
DNN_1.add(Activation('relu'))
DNN_1.add(Conv1D(32,5, strides = 1, activation='relu'))
#convLSTM_1.add(BatchNormalization())
DNN_1.add(MaxPooling1D(2))
DNN_1.add(Conv1D(16,5, activation='relu'))
DNN_1.add(MaxPooling1D(2))
DNN_1.add(Conv1D(8,5, activation='relu'))
DNN_1.add(MaxPooling1D(2))
DNN_1.add(Dropout(0.2))
DNN_1.add(Flatten())
DNN_1.add(Dense(2, activation='relu')) # Th and Tt parameters of the extended RIR model
print(DNN_1.summary())

keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.01,
    epsilon=1e-07,
    centered=False,
    name="RMSprop"
)
opt = keras.optimizers.RMSprop()


DNN_1.compile(opt, loss='mse',metrics=['mean_squared_error'])


epochs = 100
batch_size = 128
seed = 7

DNN_2 = DNN_1
DNN_3 = DNN_1
DNN_4 = DNN_1
DNN_5 = DNN_1
DNN_6 = DNN_1
DNN_7 = DNN_1


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_CNN1D (Conv1D)         (None, 181, 64)           1344      
_________________________________________________________________
activation_4 (Activation)    (None, 181, 64)           0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 177, 32)           10272     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 88, 32)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 84, 16)            2576      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 42, 16)            0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 38, 8)            

In [ ]:
history = DNN_1.fit(X_train_1, Y_train_1,validation_split=0.3, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/100
283/283 [==============================] - 9s 7ms/step - loss: 0.3542 - mean_squared_error: 0.3542 - val_loss: 0.0681 - val_mean_squared_error: 0.0681
Epoch 2/100
283/283 [==============================] - 1s 5ms/step - loss: 0.1079 - mean_squared_error: 0.1079 - val_loss: 0.1620 - val_mean_squared_error: 0.1620
Epoch 3/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0764 - mean_squared_error: 0.0764 - val_loss: 0.0420 - val_mean_squared_error: 0.0420
Epoch 4/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0613 - mean_squared_error: 0.0613 - val_loss: 0.0427 - val_mean_squared_error: 0.0427
Epoch 5/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0522 - mean_squared_error: 0.0522 - val_loss: 0.0854 - val_mean_squared_error: 0.0854
Epoch 6/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0453 - mean_squared_error: 0.0453 - val_loss: 0.0535 - val_mean_squared_error: 0.0535
Epoch 7/100
283/283 [=

In [ ]:
history2 = DNN_2.fit(X_train_2, Y_train_2,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/100
283/283 [==============================] - 2s 5ms/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.0170 - val_mean_squared_error: 0.0170
Epoch 2/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0108 - mean_squared_error: 0.0108 - val_loss: 0.0088 - val_mean_squared_error: 0.0088
Epoch 3/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0102 - mean_squared_error: 0.0102 - val_loss: 0.0092 - val_mean_squared_error: 0.0092
Epoch 4/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0098 - mean_squared_error: 0.0098 - val_loss: 0.0130 - val_mean_squared_error: 0.0130
Epoch 5/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0094 - mean_squared_error: 0.0094 - val_loss: 0.0060 - val_mean_squared_error: 0.0060
Epoch 6/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0094 - mean_squared_error: 0.0094 - val_loss: 0.0064 - val_mean_squared_error: 0.0064
Epoch 7/100
283/283 [=

In [ ]:
history3 = DNN_3.fit(X_train_3, Y_train_3,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 2/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0047 - val_mean_squared_error: 0.0047
Epoch 3/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - val_loss: 0.0058 - val_mean_squared_error: 0.0058
Epoch 4/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0068 - mean_squared_error: 0.0068 - val_loss: 0.0059 - val_mean_squared_error: 0.0059
Epoch 5/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0051 - val_mean_squared_error: 0.0051
Epoch 6/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0064 - mean_squared_error: 0.0064 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 7/100
283/283 [=

In [ ]:
history4 = DNN_4.fit(X_train_4, Y_train_4,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0051 - val_mean_squared_error: 0.0051
Epoch 2/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0060 - mean_squared_error: 0.0060 - val_loss: 0.0050 - val_mean_squared_error: 0.0050
Epoch 3/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0035 - val_mean_squared_error: 0.0035
Epoch 4/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0032 - val_mean_squared_error: 0.0032
Epoch 5/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0048 - val_mean_squared_error: 0.0048
Epoch 6/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - val_loss: 0.0042 - val_mean_squared_error: 0.0042
Epoch 7/100
283/283 [=

In [ ]:
history5 = DNN_5.fit(X_train_5, Y_train_5,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 2/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch 3/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0042 - mean_squared_error: 0.0042 - val_loss: 0.0063 - val_mean_squared_error: 0.0063
Epoch 4/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0018 - val_mean_squared_error: 0.0018
Epoch 5/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 6/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0060 - val_mean_squared_error: 0.0060
Epoch 7/100
283/283 [=

In [ ]:
history6 = DNN_6.fit(X_train_6, Y_train_6,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0022 - val_mean_squared_error: 0.0022
Epoch 2/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0060 - val_mean_squared_error: 0.0060
Epoch 3/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0031 - val_mean_squared_error: 0.0031
Epoch 4/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0022 - val_mean_squared_error: 0.0022
Epoch 5/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0032 - val_mean_squared_error: 0.0032
Epoch 6/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0027 - val_mean_squared_error: 0.0027
Epoch 7/100
283/283 [=

In [ ]:
history7 = DNN_7.fit(X_train_7, Y_train_7,validation_split=0.3,
            batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 2/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0031 - val_mean_squared_error: 0.0031
Epoch 3/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0018 - val_mean_squared_error: 0.0018
Epoch 4/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0027 - val_mean_squared_error: 0.0027
Epoch 5/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0046 - val_mean_squared_error: 0.0046
Epoch 6/100
283/283 [==============================] - 1s 5ms/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0026 - val_mean_squared_error: 0.0026
Epoch 7/100
283/283 [=

In [ ]:
DNN_1.save("MTFbasedCNN_ext_1.h5")
DNN_2.save("MTFbasedCNN_ext_2.h5")
DNN_3.save("MTFbasedCNN_ext_3.h5")
DNN_4.save("MTFbasedCNN_ext_4.h5")
DNN_5.save("MTFbasedCNN_ext_5.h5")
DNN_6.save("MTFbasedCNN_ext_6.h5")
DNN_7.save("MTFbasedCNN_ext_7.h5")


Done!!
